In [3]:
import numpy as np

In [4]:
data_dir = '/data/xd/TS_麻醉/数据/TRAINDATA_BBF_RFTN_ASA_DIAGNOSE'
train_txt_path = '/data/xd/01_My_RL_Mazui_Model/V2_anethesia/train_files.txt'
valid_txt_path = '/data/xd/01_My_RL_Mazui_Model/V2_anethesia/valid_files.txt'

In [5]:
def read_filename_from_txt(file_path:str):
    file_name:list = []
    
    try:
        with open(file_path, 'r') as f:
            for line in f:
                file_name.append(line.strip())
    
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        
    except Exception as e:
        print(f"Error: {e}")
        
    return file_name

train_file_name:list = read_filename_from_txt(train_txt_path)
valid_file_name:list = read_filename_from_txt(valid_txt_path)

In [6]:
import os
train_data_path:list = []
data_dir = '/data/xd/TS_麻醉/数据/TRAINDATA_BBF_RFTN_ASA_DIAGNOSE/'
for i in train_file_name:
    train_data_path.append(os.path.join(data_dir, i))

valid_data_path:list = []
for i in valid_file_name:
    valid_data_path.append(os.path.join(data_dir, i))

print(train_data_path.__len__(), valid_data_path.__len__())

961 107


In [7]:
print(train_file_name.__len__(), valid_file_name.__len__())

961 107


In [8]:
import pandas as pd
from RL.Module.data import TimeseriesExtractor, ActionExtractor
from RL.pip import *
from RL.agent import ArmAgent
from torch.utils.data.dataloader import default_collate
from constant import *

In [9]:
def get_sample(df,df_meta_path,_map,gamma):

    feat_extractor = TimeseriesExtractor(df_meta_path,
                                         add_mask=add_mask)
    # 打药执行序列
    action_extractor = ActionExtractor()
    
    #把df下标从max_length到

    #建立索引
    df_case_raw = df.reset_index(drop=True)
    # 建立新列,名字为time,为索引1,2,3..
    df_case_raw['timestamps'] = np.arange(len(df_case_raw))
    gamma = float(gamma)
    # extract features
    df_case_feat = feat_extractor.transform(df_case_raw)
    df_obs = df_case_feat
    # 把label_*列的值找出来,汇集成列表
    obs = df_obs.fillna(0).values
    l = len(obs)
    df_aux = pd.DataFrame({'Column': [1] * l})
    # 找出缺失表,把缺失的值填充为0,不缺失的值填充为1
    df_mask_aux = df_aux.notna().astype('int')
    # get the value,对nan值进行填充
    # get the value,对nan值进行填充
    obs = df_obs.fillna(0).values

    aux = df_aux.fillna(0).values
    mask_aux = df_mask_aux.values
    result = {
        'obs': obs,
        'aux': aux,
        'mask_aux': mask_aux,
    }

    df_case_raw['reward'] = cal_reward(df_case_feat[cols_state], dtype='risk',f_map=_map)
    df_case_raw['return'] = reward2return(df_case_raw['reward'], gamma=gamma)
    df_case_raw = action_extractor.transform(df_case_raw)  # add action and option

    df_reward = df_case_raw['reward']
    df_return = df_case_raw['return']
    df_action = (df_case_raw['action_BBF'], df_case_raw['action_RFTN'])
    df_option = df_case_raw['option']
    # 填充值
    reward = df_reward.fillna(0).values
    cumreward = df_return.fillna(0).values
    action_BBF = df_action[0].fillna(0).values
    action_RFTN = df_action[1].fillna(0).values
    option = df_option.fillna(0).values

    df_mask_action = (df_action[0] > 0).astype('int')
    mask_action = df_mask_action.values

    bis = df_case_feat['BIS'].values
    rp= df_case_feat['MAP|MAP'].values

    result.update({
            'reward': reward,
            'cumreward': cumreward,
            'action': (action_BBF, action_RFTN),
            'option': option,
            'mask_action': mask_action,
            'BIS': bis,#生命体征
            'MAP|MAP':rp
    })

        # padding: [:l] is False, [l:] is True
    l = len(obs)
    padding = np.zeros(l, dtype='bool')
   #对于


    return result

In [10]:
def dataframe2tensor(sample):


    #获取tensor
    # 在前面部分获取sample(包括obs,aux,mask_aux,padding,live,reward,cumreward,action,option,mask_action)
    obs = sample['obs'].astype('float32')
    aux = sample['aux'].astype('float32')  # for BCE loss
    aux = np.roll(aux, -1, axis=0)
    aux[-1] = 0
    mask_aux = sample['mask_aux'].astype('int')
    mask_aux = np.roll(mask_aux, -1, axis=0)
    mask_aux[-1] = 0
    bis = sample['BIS'].astype('int')
    BIS_target = bis # move forward one point of time as a label

    rp = sample['MAP|MAP'].astype('int')
    rp_target = rp  # move forward one point of time as a label


    reward = sample['reward'].astype('float32')


    cumreward = sample['cumreward'].astype('float32')


    action_BBF = sample['action'][0].astype('int')
    action_RFTN = sample['action'][1].astype('int')
    action_BBF = action_BBF.clip(0, n_action)#对action切片
    action_RFTN = action_RFTN.clip(0, n_action)#对action切片
    
    BIS_target= BIS_target.clip(0, n_bis)
    rp_target=rp_target.clip(0, n_rp)
    
    action_prev = (np.roll(action_BBF, shift=1), np.roll(action_RFTN, shift=1))  # 空出第一个位置
    action_prev[0][0] = 0
    action_prev[1][0] = 0
    action=action_prev
    
    option = sample['option'].astype('int')
    option = option.clip(0, 4)
    mask_reward = bis > 0.01
    mask_reward[-1] = False



    return (obs, action_prev, option, BIS_target,rp_target)  # 二维

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

class Model(object):
    def __init__(self, model_dir, df_meta_path, beam_size=2, device=None):
        super(Model, self).__init__()
        self.df_meta_path = df_meta_path
        self.beam_size = beam_size
        self.model_dir = model_dir

        self.agent = ArmAgent.build_from_dir(model_dir, device)
        self.size = self.agent.max_len
    
    def predict_v1(self,df_meta_path, df, beam_size=None,name=None):
        first_map=df["MAP|MAP"][0]
        end_index=start_len
        import torch

        # 决策与预测产生的四种结果
        df_pred = pd.DataFrame()
        df_pred_ac_t_bbf=torch.tensor([0, 0, 0, 0,0, 0, 0, 0, 0, 0])
        df_pred_ac_t_rftn=torch.tensor([0, 0, 0, 0,0, 0, 0, 0, 0, 0])
        df_pred_bis_t=torch.tensor([0, 0, 0, 0,0, 0, 0, 0, 0, 0])
        df_pred_pr_t=torch.tensor([0, 0, 0, 0,0, 0, 0, 0, 0, 0])
        
        # 对应model输出的真实记录
        df_target = pd.DataFrame()
        df_target['action_BBF']=df['BBF_SPEED']
        df_target['action_RFTN']=df['RFTN_SPEED']
        df_target['bis']=df['BIS']
        df_target['map']=df['MAP|MAP']
        df_target['action_BBF'] = np.roll( df_target['action_BBF'], shift=1)  # 空出第一个位置
        df_target['action_BBF'][0] = 0
        df_target['action_RFTN'] = np.roll( df_target['action_RFTN'], shift=1)  # 空出第一个位置
        df_target['action_RFTN'][0] = 0
        padding = np.zeros(len(df), dtype='bool')
        
        
        for i in range(end_index, len(df), pre_len):
            if(i<max_lenth):
                true_data=df.iloc[0:i].copy()
                true_data = pd.concat([true_data,pd.DataFrame(index=range(max_lenth-i))], axis=0)#随机填充,增强对开始任务的学习能力量

                selected_padding= np.copy(padding[0:i])
                selected_padding =  np.concatenate([ selected_padding,np.ones(max_lenth-i, dtype='bool')])

            else:
                true_data=df.iloc[i-max_lenth:i].copy()
                selected_padding= np.copy(padding[i-max_lenth:i])
            

   
        
            sample = get_sample(true_data,df_meta_path,first_map,gamma)
            r=[]
            # 例如，打印每个窗口的内容
            tensors = dataframe2tensor(sample)
            tensors = default_collate([tensors])#堆叠张量
            obs, action, option, BIS_target,rp_target = tensors

            # convert to tensor
            # predict
            beam_size = beam_size if beam_size is not None else self.beam_size
            # 记录开始时间
            import time
            start_time = time.time()
            output = self.agent.self_rollout(obs, action, option,selected_padding, beam_size=beam_size)  # extra one day
            end_time = time.time()
            execution_time = end_time - start_time
            # print(f"代码执行花费的时间: {execution_time}秒")
            actions_out, bis_out,rp_out = output
            
            mean_dose_BBF =torch.mean(actions_out[0].float())
            mean_dose_BBF =torch.full(size=(1,5),fill_value=mean_dose_BBF)
            mean_dose_RFTN =torch.mean(actions_out[1].float())
            mean_dose_RFTN =torch.full(size=(1,5),fill_value=mean_dose_RFTN)
            
            #padding_series = pd.Series([0] * max_lenth)
            df_pred_ac_t_bbf= torch.cat([df_pred_ac_t_bbf , mean_dose_BBF[0,:]], dim=0)
            df_pred_ac_t_rftn= torch.cat([df_pred_ac_t_rftn , mean_dose_RFTN[0,:]], dim=0)

            df_pred_bis_t=torch.cat([df_pred_bis_t ,  bis_out[0,:]], dim=0)
            df_pred_pr_t=torch.cat([df_pred_pr_t ,  rp_out[0,:]], dim=0)
        
        df_pred['action_BBF']=df_pred_ac_t_bbf
        df_pred['action_RFTN']=df_pred_ac_t_rftn
        df_pred['bis']=df_pred_bis_t
        df_pred['map']=df_pred_pr_t
        
        
         # 计算 MAE 和 MSE
        # 确保预测值和真实值的长度一致
        min_length = min(len(df_pred), len(df_target))
        df_pred = df_pred.iloc[:min_length]
        df_target = df_target.iloc[:min_length]

        # 计算 MAE 和 MSE
        mae_action_BBF = mean_absolute_error(df_target['action_BBF'][10:], 
                                             df_pred['action_BBF'][10:])
        mse_action_BBF = mean_squared_error(df_target['action_BBF'][10:],
                                            df_pred['action_BBF'][10:])

        mae_action_RFTN = mean_absolute_error(df_target['action_RFTN'][10:], 
                                              df_pred['action_RFTN'][10:])
        mse_action_RFTN = mean_squared_error(df_target['action_RFTN'][10:],
                                             df_pred['action_RFTN'][10:])

        mae_bis = mean_absolute_error(df_target['bis'][10:], 
                                      df_pred['bis'][10:])
        mse_bis = mean_squared_error(df_target['bis'][10:], 
                                     df_pred['bis'][10:])

        mae_map = mean_absolute_error(df_target['map'][10:], 
                                      df_pred['map'][10:])
        mse_map = mean_squared_error(df_target['map'][10:], 
                                     df_pred['map'][10:])

    
        # 将统计结果保存到 CSV 文件
        results = {
            'Metric': ['MAE', 'MSE'],
            'Action_BBF': [mae_action_BBF, mse_action_BBF],
            'Action_RFTN': [mae_action_RFTN, mse_action_RFTN],
            'BIS': [mae_bis, mse_bis],
            'MAP': [mae_map, mse_map],
        }
        
        # 打印结果
        print(f"MAE - Action BBF: {mae_action_BBF}")
        print(f"MSE - Action BBF: {mse_action_BBF}")
        print(f"MAE - Action RFTN: {mae_action_RFTN}")
        print(f"MSE - Action RFTN: {mse_action_RFTN}")
        print(f"MAE - BIS: {mae_bis}")
        print(f"MSE - BIS: {mse_bis}")
        print(f"MAE - MAP: {mae_map}")
        print(f"MSE - MAP: {mse_map}")

        
        return results
        

In [12]:
import warnings

# 忽略所有警告
warnings.filterwarnings('ignore')

In [13]:
model = Model(model_dir=model_dir, df_meta_path=data_kargs_default['df_meta_path'])

In [14]:
import glob

# 设置文件夹路径和文件模式
folder_path = "/data/xd/01_My_RL_Mazui_Model/V2_anethesia/anesthesia/量化结果/valid"
csv_files = glob.glob(folder_path + "/*.csv")

# 输出 .csv 文件的个数
print(f"文件夹下有 {len(csv_files)} 个 .csv 文件")

文件夹下有 0 个 .csv 文件


In [15]:
# 遍历每个 CSV 文件并进行测试
for csv_file in valid_data_path:

    # 读取 CSV 文件
    df_data = pd.read_csv(csv_file)
    if df_data.__len__() > 15:
        # 调用 predict_v1 方法
        print(f"正在处理文件: {csv_file}")
        result = model.predict_v1(df=df_data, df_meta_path=data_kargs_default['df_meta_path'], beam_size=5, name=csv_file)
        results_df = pd.DataFrame(result)
        results_df.to_csv('/data/xd/01_My_RL_Mazui_Model/V2_anethesia/anesthesia/量化结果/valid/'+os.path.basename(csv_file), index=False)
        print(f"文件 {csv_file} 处理完成\n")

正在处理文件: /data/xd/TS_麻醉/数据/TRAINDATA_BBF_RFTN_ASA_DIAGNOSE/982.csv
MAE - Action BBF: 0.41697418909671125
MSE - Action BBF: 1.8811808624830668
MAE - Action RFTN: 0.6398523988758946
MSE - Action RFTN: 4.166051718933563
MAE - BIS: 1.9901599015990157
MSE - BIS: 7.791717917179172
MAE - MAP: 4.528290282902829
MSE - MAP: 38.07287822878229
文件 /data/xd/TS_麻醉/数据/TRAINDATA_BBF_RFTN_ASA_DIAGNOSE/982.csv 处理完成

正在处理文件: /data/xd/TS_麻醉/数据/TRAINDATA_BBF_RFTN_ASA_DIAGNOSE/118.csv
MAE - Action BBF: 0.6818180734461005
MSE - Action BBF: 0.8409087874673044
MAE - Action RFTN: 1.4999998699535022
MSE - Action RFTN: 4.00909028486777
MAE - BIS: 3.6325757575757573
MSE - BIS: 16.505050505050505
MAE - MAP: 8.09469696969697
MSE - MAP: 102.3838383838384
文件 /data/xd/TS_麻醉/数据/TRAINDATA_BBF_RFTN_ASA_DIAGNOSE/118.csv 处理完成

正在处理文件: /data/xd/TS_麻醉/数据/TRAINDATA_BBF_RFTN_ASA_DIAGNOSE/392.csv
MAE - Action BBF: 0.37053571215697695
MSE - Action BBF: 1.7044642831598011
MAE - Action RFTN: 0.325892835855484
MSE - Action RFTN: 1.191